## add scores to retriever
---
在檢索系統(retriever)的搜索結果中加入相似性分數有以下幾點重要好處：
+ 質量評估：分數能幫助評估每個文檔與查詢的相關程度，讓你能夠評估檢索質量。
+ 排序和過濾：可以實施基於閾值的過濾，排除低相關性文檔或按相關性排序結果。
+ 透明度：通過顯示為什麼某些文檔被檢索出來，使檢索過程更加可解釋。
+ 信心估計：幫助判斷在生成回應時對檢索信息的信任程度。
+ 調試功能：在構建RAG應用程序時，如果結果不如預期，相似性分數提供關鍵的診斷信息。
+ 更好的提示工程：可以在提示中包含相關性分數，幫助LLM優先考慮更相關的信息。
+ 用戶體驗：在面向用戶的應用中，這些分數可以顯示給用戶，表明檢索信息的可信度。

在MultiVectorRetriever的例子中，跟踪子文檔的分數特別有價值，因為它顯示了哪些特定文本片段觸發了父文檔的檢索，提供了對檢索過程更深入的洞察。

---
以下先建立測試用的 Chroma 向量數據庫，然後使用 `similarity_search_with_score` 方法來獲取相似性分數。接著，將這些分數添加到檢索器的結果中，以便在後續的處理中使用。

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="onnxruntime") #windows2016 < widows10
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

from env_properties import read_properties

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

properties = read_properties("../config.properties")
openai_api_key = properties.get("openai_api_key")
vectorstore = Chroma.from_documents(docs, embedding=OpenAIEmbeddings(model="text-embedding-3-small", api_key=openai_api_key))

## Retriever

為了獲取分數，使用 `similarity_search_with_score` 方法。這樣可以獲取每個檢索到的文檔的相似性分數。
加了 `@chain` 裝飾器的函數會創建一個 [Runnable](/docs/concepts/lcel)，可以像典型的檢索器一樣使用。


In [2]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score
    return docs
result = retriever.invoke("dinosaur")
print(result)

(Document(id='c1eef9f0-b43f-4a61-bdb7-10f1ba98e4c4', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993, 'score': 0.9784544110298157}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'), Document(id='37831d76-523e-4af5-afc8-4bcfb7094fec', metadata={'genre': 'animated', 'year': 1995, 'score': 1.503709316253662}, page_content='Toys come alive and have a blast doing so'), Document(id='8d299662-b6a6-4587-b3a9-ec6b64e13f63', metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010, 'score': 1.6437551975250244}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'), Document(id='5f264fd4-9690-4f14-9252-e54b000da83b', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006, 'score': 1.7278307676315308}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'))


## SelfQueryRetriever

`SelfQueryRetriever` 將使用 LLM 生成一個潛在結構化的查詢--例如，它可以在通常基於語義相似性驅動的選擇之上構建檢索過濾器。請參閱 `self_query.ipynb` 獲取更多詳細信息。

`SelfQueryRetriever` 包含了一個簡短的 (1 - 2 行) 方法 `_get_docs_with_query`，該方法執行 `vectorstore` 搜索。我們可以子類化 `SelfQueryRetriever` 並覆蓋此方法以傳播相似性分數。

首先，我們需要按照 `self_query.ipynb` 建立一些元數據來進行過濾：


In [3]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

將 `_get_docs_with_query` 方法覆蓋到 `SelfQueryRetriever` 中，使用 `similarity_search_with_score` 方法來獲取相似性分數。這樣，我們可以在檢索到的文檔中包含這些分數，並在後續的處理中使用它們。

啟動這個檢索器時，將會在文檔的元數據中包含相似性分數。請注意，`SelfQueryRetriever` 的底層結構化查詢功能仍然保留。


In [4]:
from typing import Any, Dict
from langchain.retrievers.self_query.base import SelfQueryRetriever

class CustomSelfQueryRetriever(SelfQueryRetriever):
    def _get_docs_with_query(
        self, query: str, search_kwargs: Dict[str, Any]
    ) -> List[Document]:
        """Get docs, adding score information."""
        docs, scores = zip(
            *self.vectorstore.similarity_search_with_score(query, **search_kwargs)
        )
        for doc, score in zip(docs, scores):
            doc.metadata["score"] = score
        return docs

retriever = CustomSelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)
result = retriever.invoke("dinosaur movie with rating less than 8")
print(result)

(Document(id='c1eef9f0-b43f-4a61-bdb7-10f1ba98e4c4', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993, 'score': 0.9784544110298157}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),)


## MultiVectorRetriever

`MultiVectorRetriever`允許您將多個向量與單個文檔關聯。這在許多應用中都很有用。例如，我們可以索引較大文檔的小塊，並在塊上運行檢索，但在調用檢索器時返回較大的“父”文檔。 (見parent_document_retriever.ipynb)/)，`MultiVectorRetriever` 的子類，包含填充向量存儲以支持此操作的便捷方法。 進一步的應用程序詳細說明見 multi_vector.ipynb

經由這個檢索器檢索的文檔將包含與其相關的子文檔的分數。這在許多應用中都很有用。例如，我們可以索引較大文檔的小塊，並在塊上運行檢索，但在調用檢索器時返回較大的“父”文檔。 (見parent_document_retriever.ipynb)/)，`MultiVectorRetriever` 的子類，包含填充向量存儲以支持此操作的便捷方法。 進一步的應用程序詳細說明見 multi_vector.ipynb

首先，我們準備一些測試數據。我們生成假“整個文檔”，並將它們存儲在文檔存儲中；這裡我們將使用一個簡單的 [InMemoryStore](https://python.langchain.com/api_reference/core/stores/langchain_core.stores.InMemoryBaseStore.html)。

然後加上一些假的"sub-documents" 到vector store。我們可以連結這些子文檔到父文檔，通過填充它的元數據中的 `"doc_id"` 鍵值。


In [5]:
from langchain.storage import InMemoryStore

# The storage layer for the parent documents
docstore = InMemoryStore()
fake_whole_documents = [
    ("fake_id_1", Document(page_content="fake whole document 1")),
    ("fake_id_2", Document(page_content="fake whole document 2")),
]
docstore.mset(fake_whole_documents)

docs = [
    Document(
        page_content="A snippet from a larger document discussing cats.",
        metadata={"doc_id": "fake_id_1"},  # This is the ID of the parent document
    ),
    Document(
        page_content="A snippet from a larger document discussing discourse.",
        metadata={"doc_id": "fake_id_1"},  # This is the ID of the parent document
    ),
    Document(
        page_content="A snippet from a larger document discussing chocolate.",
        metadata={"doc_id": "fake_id_2"},  # This is the ID of the parent document
    ),
]
vectorstore.add_documents(docs)

['11cbaf95-fc10-4053-bc7b-29324532a69e',
 'c0bfc8ca-1fbb-4e3f-a1f4-53313f712bc1',
 'f4c0762e-b228-4868-9421-ef970cdd9859']

為了帶上這些相似性分數，我們將繼承 `MultiVectorRetriever` 並覆蓋其 `_get_relevant_documents` 方法。我們將進行兩個更改：

1. 如上所述, 我們將使用 `similarity_search_with_score` 方法將相似性分數添加到相應的“子文檔”的元數據中
2. 我們將在檢索的父文檔的元數據中包含這些子文檔的列表。這樣可以顯示哪些文本片段被檢索到，以及它們相應的相似性分數。

In [6]:
from collections import defaultdict

from langchain.retrievers import MultiVectorRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

class CustomMultiVectorRetriever(MultiVectorRetriever):
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Get documents relevant to a query.
        Args:
            query: String to find relevant documents for
            run_manager: The callbacks handler to use
        Returns:
            List of relevant documents
        """
        results = self.vectorstore.similarity_search_with_score(
            query, **self.search_kwargs
        )

        # Map doc_ids to list of sub-documents, adding scores to metadata
        # defaultdict會創建一個默認字典，存取不存在的key值不會出錯
        id_to_doc = defaultdict(list)
        for doc, score in results:
            doc_id = doc.metadata.get("doc_id")
            if doc_id:
                doc.metadata["score"] = score
                id_to_doc[doc_id].append(doc)

        # Fetch documents corresponding to doc_ids, retaining sub_docs in metadata
        docs = []
        for _id, sub_docs in id_to_doc.items():
            docstore_docs = self.docstore.mget([_id])
            if docstore_docs:
                if doc := docstore_docs[0]:
                    doc.metadata["sub_docs"] = sub_docs
                    docs.append(doc)
        return docs

retriever = CustomMultiVectorRetriever(vectorstore=vectorstore, docstore=docstore)
result = retriever.invoke("cat")
print(result)


[Document(metadata={'sub_docs': [Document(id='11cbaf95-fc10-4053-bc7b-29324532a69e', metadata={'doc_id': 'fake_id_1', 'score': 1.1385878324508667}, page_content='A snippet from a larger document discussing cats.')]}, page_content='fake whole document 1')]
